In [2]:
# Load dataframe of CVPR+citing papers, authors, instiutions, and relations to patents with 'surveil'

import compress_pickle as pickle
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

paper_info_df=pickle.load('/gscratch/prl/wagnew3/microsoft_academic_graph/cvpr_procssed_dataset.lz4')

save_loc="/gscratch/prl/wagnew3/microsoft_academic_graph/"
conf_save_path=os.path.join(save_loc, 'mag_conf_journ_title.lz4')
mag_to_conf_id, mag_to_journal_id, mag_to_title=pickle.load(conf_save_path)

df_mag_to_conf_id=pd.DataFrame.from_dict(mag_to_conf_id, orient='index')
df_mag_to_conf_id.index.name = 'mag'
df_mag_to_conf_id.reset_index(inplace=True)
df_mag_to_conf_id.columns=['mag', 'conf_if']
print(df_mag_to_conf_id.columns)
paper_info_df=paper_info_df.merge(df_mag_to_conf_id, left_on='paper_id', right_on='mag', how='left')

keyword_depth_vecs=paper_info_df["keyword_depth_vec"].to_list()
keyword_depth_vecs=[np.reshape(keyword_depth_vecs[i], (-1,)) for i in range(len(keyword_depth_vecs))]


Index(['mag', 'conf_if'], dtype='object')


In [4]:
conf_save_path=os.path.join(save_loc, 'temp_keywords_cvpr.lz4')
print(conf_save_path)
mag_id_keyword_depth_vecs, stored_patents_keywords_dict=pickle.load(open(conf_save_path, 'rb'))

/gscratch/prl/wagnew3/microsoft_academic_graph/temp_keywords_cvpr.lz4


In [16]:
patents_info_dict={}
agg_data_dir="/gscratch/scrubbed/wagnew3/data/"
for agg_file in tqdm(os.listdir(agg_data_dir)):
    if "patent_agg" in agg_file:
        patent_info_sub=pickle.load(os.path.join(agg_data_dir, agg_file))
        patents_info_dict.update(patent_info_sub)
print(f"{len(patents_info_dict)} patents")

100%|█████████████████████████████████████████████| 26/26 [03:28<00:00,  8.03s/it]

2549890 patents


In [20]:
print(patents_info_dict['US8965141'])

['US8965141', 'Image filtering based on structural information', '[{"inventor_name": "Wen Fei JIANG"}, {"inventor_name": "Fan Zhang"}, {"inventor_name": "Zhi Bo Chen"}]', '[{"assignee_name": "Thomson Licensing SAS"}]', '[{"assignee_name": "\\n    Magnolia Licensing LLC\\n  "}]', '2014-02-20', '2012-08-17', '2015-02-24', '', '\n     A method and an apparatus for image filtering are described. Structural information is employed during the calculation of filtering coefficients. The structural information is described by the regions defined through an edge map of the image. In one embodiment, the region correlation between the target pixel and a contributing pixel is selected as a structural filtering coefficient. The region correlation, which indicates the possibility of two pixels being in the same regions, is calculated by evaluating the strongest edge cut by a path between the target pixel and a contributing pixel. The structural filtering coefficient is further combined with spatial i

In [15]:
print(stored_patents_keywords_dict['2161969291']['person'])

# for k in stored_patents_keywords_dict:
#     print(stored_patents_keywords_dict[k])
#     break

['US9996917', 'US9996554', 'US9992467', 'US9984315', 'US9977980', 'US9977955', 'US9977816', 'US9971934', 'US9965728', 'US9946933', 'US9922265', 'US9913135', 'US9911053', 'US9898701', 'US9886761', 'US9886653', 'US9881214', 'US9875213', 'US9852326', 'US9846429', 'US9842274', 'US9835792', 'US9830527', 'US9824462', 'US9798959', 'US9792899', 'US9785828', 'US9779331', 'US9767348', 'US9760794', 'US9754188', 'US9740034', 'US9734392', 'US9721192', 'US9715902', 'US9715639', 'US9710716', 'US9710447', 'US9710109', 'US9709723', 'US9699419', 'US9697442', 'US9679370', 'US9679201', 'US9672634', 'US9665777', 'US9659384', 'US9659238', 'US9659225', 'US9652663', 'US9646212', 'US9639783', 'US9639777', 'US9639762', 'US9633284', 'US9633264', 'US9626551', 'US9582895', 'US9576371', 'US9576196', 'US9571723', 'US9569694', 'US9569531', 'US9569498', 'US9567078', 'US9552510', 'US9547791', 'US9542621', 'US9530080', 'US9504420', 'US9501710', 'US9489570', 'US9483838', 'US9471171', 'US9465998', 'US9449245', 'US9445763'

In [3]:
patent_dists=pd.DataFrame(keyword_depth_vecs, columns=['0_dist', '1_dist', '2_dist', '3_dist', '0_other', '1_other', '2_other', '3_other'])
df = pd.concat([paper_info_df, patent_dists], axis=1) 
df=df.drop(columns=['keyword_depth_vec_affils', 
                 'keyword_depth_vec_author',
                'PaperId_author',
                'AuthorId_author',
                'AffiliationId_author',
                'AffiliationId_author',
                'PaperId_fields',
                 'keyword_depth_vec_fields'])

print(df.iloc[0])
# https://learn.microsoft.com/en-us/academic-services/graph/reference-data-schema gives you most information about what is in the dataframe df. 
# "keyword_depth_vec" is a 2x4 numpy array, where the first row is the number of "surveil" patent cites at 0,1,2,3 paper citation depth, 
# and the second row is the number of non-"surveil" patent cites. 
# This is broken into columns '0_dist', '1_dist', '2_dist', '3_dist' (surveil patent cites) 
# '0_other', '1_other', '2_other', '3_other' (non surveil patent cites)


def implode(df, column):
    agg_keys={}
    for key in df.columns:
        if key!=column:
            agg_keys[key]=list
            
    return df.groupby(column, as_index=False).agg(agg_keys)[df.columns]

ValueError: 8 columns passed, passed data had 6 columns

In [ ]:
# Load citating graph for CVPr+citing papers. Takes a long time, only needed for river charts, not bar charts

cvpr_graph=pickle.load(open(os.path.join(save_loc, f'cvpr_graph_-1.lz4'), 'rb'))
expanded_cvpr_graph={}
for k,v in tqdm(cvpr_graph.items()):
    expanded_cvpr_graph[k]=pd.Series([v])
cvpr_graph_df=pd.DataFrame(expanded_cvpr_graph).T
print(cvpr_graph_df)

In [ ]:
# Draft river chart function

import plotly.graph_objects as go
import plotly.graph_objects as go
import textwrap
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import rgb2hex

def river_chart(df, first_col, first_col_filter, group_col, ptp_df, backward=False, max_link_ind=4, num_per_link=10):
    # Get matching entities
    print('df.columns', df.columns)

    e_df=df.explode(first_col)
    if len(first_col_filter)==0:
        past_entities=e_df[e_df['conf_if']=='1158167855']
    else:
        past_entities=e_df[(e_df[first_col] == first_col_filter) & (e_df['conf_if']=='1158167855')]
    # past_entities=implode(past_entities, 'paper_id')

    dist_list=['0_dist', '1_dist', '2_dist', '3_dist']


    print('len(past_entities)', len(past_entities))
    
    df_mag=df.set_index('paper_id')
    past_entities=past_entities.set_index('paper_id')
    past_entity_names=past_entities[first_col].unique()
    top_past_entity_names=past_entity_names[:num_per_link].tolist()


    # river chart parts
    source=[]
    target=[]
    value=[]
    color=[]
    labels=[]

    river_ind=0
    name_to_river_ind={}


    past_other_2=[]

    wrapper=textwrap.TextWrapper(width=20)

    # Get down/upstream citations
    for link_ind in range(0, max_link_ind):

        cur_other=[]
        past_other=[]

        if link_ind==0:
            past_entry_mags=past_entities.index.tolist()
            citing_magss=[]
            for past_entry_mag in past_entry_mags:
                if past_entry_mag in ptp_df.index:
                    citing_magss.append(past_entry_mag)
        else:
            past_entry_mags=past_entities.index.tolist()
            citing_magss=[]
            for past_entry_mag in past_entry_mags:
                if past_entry_mag in ptp_df.index:
                    citing_magss+=ptp_df.loc[str(past_entry_mag)].tolist()[0]

        dist_key=dist_list[len(dist_list)-link_ind-1]
        in_citing_magss=citing_magss

        print('len(in_citing_magss)', len(in_citing_magss))
        citing_cols=df_mag.loc[in_citing_magss]
        citing_cols=citing_cols[['PaperId', group_col, 'keyword_depth_vec']]
        citing_cols['keyword_depth_vec']=citing_cols['keyword_depth_vec'].apply(lambda x: x[0][:max_link_ind-max(0,link_ind)].sum())
        print('reduce citing_cols.columns', citing_cols.columns)
        citing_cols=citing_cols.explode(group_col)
        citing_cols=implode(citing_cols, group_col)
        citing_cols['keyword_depth_vec']=citing_cols['keyword_depth_vec'].apply(lambda x: sum(x))
        citing_cols=citing_cols.explode('PaperId')
        citing_cols=citing_cols.sort_values('keyword_depth_vec', ascending=False)
        # print('citing_cols', citing_cols)

        # Compute flows between past and current cite level
        #print('past_entity_names[:num_per_link]', top_past_entity_names)
        top_cur_names=citing_cols[group_col].unique()[:num_per_link].tolist()
        print('top_cur_names', top_cur_names)
        # print(top_cur_names)
        links={}
        print('len(in_citing_magss)', len(in_citing_magss))
        citing_df=df_mag.loc[in_citing_magss]
        print('len(citing_df)', len(citing_df['keyword_depth_vec']))
        unique_citing_df=df_mag.loc[set(in_citing_magss)]
        cited_df=df_mag.loc[past_entities.index]
        list_keyword_depth_vec=citing_df['keyword_depth_vec'].tolist()
        list_group_col=citing_df[group_col].tolist()                     
        citing_mag_ind=0
        surveil_added={}
        for mag_ind in tqdm(range(len(past_entity_names))):
            mag=past_entities.index[mag_ind]
            sub_past_entities=past_entity_names[mag_ind]

            if not isinstance(sub_past_entities, list):
                sub_past_entities=[sub_past_entities]
            sub_past_entities=set(sub_past_entities)

            if mag in ptp_df.index:
                #mag_keyword_depth_vec=cited_df.loc[mag]['keyword_depth_vec']
                if link_ind==0:
                    citing_mags=citing_df[citing_df[first_col]==past_entity_names[mag_ind]].index
                else:
                    citing_mags=ptp_df.loc[mag][0]
                    # if mag_ind==0:
                    #     print('citing_mags', citing_mags)
                #citing_mags=list(set(citing_mags))
                #citing_mags.append('surveil patent')
                #citing_infoss=unique_citing_df.loc[citing_mags]
                for citing_mag in citing_mags:
                    between_cites=list_keyword_depth_vec[citing_mag_ind][0][:max_link_ind-max(0,link_ind)].sum()#citing_info['keyword_depth_vec'][0][:max_link_ind-max(0,link_ind)].sum()#
                    other_between_cites=list_keyword_depth_vec[citing_mag_ind][1][:max_link_ind-max(0,link_ind)].sum()

                    direct_between_cites=list_keyword_depth_vec[citing_mag_ind][0][0]
                    direct_other_between_cites=list_keyword_depth_vec[citing_mag_ind][1][0]

                    current_entities=list_group_col[citing_mag_ind]#citing_info[group_col]#
                    if not isinstance(current_entities, list):
                        current_entities=[current_entities]
                    current_entities=set(current_entities)

                    citing_mag_ind+=1
                    
                    

                    for past_entity in sub_past_entities:
                        for current_entity in current_entities:
                            if past_entity in top_past_entity_names:
                                past_name=past_entity
                            else:
                                past_name=f'other_{link_ind}'

                            if current_entity in top_cur_names:
                                cur_name=current_entity
                            else:
                                cur_name=f'other_{link_ind+1}'

                            if past_name not in links:
                                links[past_name]={}
                            if cur_name not in links[past_name]:
                                links[past_name][cur_name]=np.zeros(2)
                            links[past_name][cur_name][0]+=between_cites
                            links[past_name][cur_name][1]+=other_between_cites

                            # Add links to surveillance patents
                            if cur_name not in links:
                                links[cur_name]={}
                            if 'surveil patent' not in links[cur_name]:
                                links[cur_name]['surveil patent']=np.zeros(2)
                            # if current_entity not in surveil_added:
                            links[cur_name]['surveil patent'][0]+=direct_between_cites
                            links[cur_name]['surveil patent'][1]+=direct_other_between_cites
        # print(links)
                    # else:
                    #     print('not in', citing_mag)

        # print('past_other', past_other)
        # print('cur_other', cur_other)
        past_other_2=cur_other

        # Add flows to graph
        for past_entity in links:
            for current_entity in links[past_entity]:
                # Get ind for prev river entry
                if past_entity not in name_to_river_ind:
                    name_to_river_ind[past_entity]=river_ind
                    labels.append(past_entity)
                    river_ind+=1

                # Get ind for current river entry
                if current_entity not in name_to_river_ind:
                    name_to_river_ind[current_entity]=river_ind
                    labels.append(current_entity)
                    river_ind+=1

                # Add data to river chart info
                source.append(name_to_river_ind[past_entity])
                target.append(name_to_river_ind[current_entity])
                value.append(links[past_entity][current_entity][0])
                if links[past_entity][current_entity][0]+links[past_entity][current_entity][1]>0:
                    color.append(links[past_entity][current_entity][0]/(links[past_entity][current_entity][0]+links[past_entity][current_entity][1]))
                else:
                    color.append(0)


        past_entity_names=citing_df[group_col]#.unique()
        top_past_entity_names=top_cur_names
        # print('past_entity_names', past_entity_names)
        past_entities=citing_df

    color=np.array(color)
    cmap = plt.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
    colors=cmap(norm(color))
    colors_hex=[rgb2hex(color) for color in colors]

    colors_transparent=[f'rgba(0,0,0,{opacity})' for opacity in color]

    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = labels,
        ),
        link = dict(
          source = source,
          target = target,
          value = value,
          color = colors_transparent
      ))])

    fig.update_layout(title_text="Basic Sankey Diagram", font_size=10, width=2000, height=1500)
    fig.show()

river_chart(df, 'NormalizedName_author', 'li feifei', 'PaperTitle', cvpr_graph_df, backward=False, max_link_ind=1)
#river_chart(df, 'NormalizedName_author', '', 'PaperTitle', cvpr_graph_df, backward=True, max_link_ind=2)
    

In [ ]:
# Draft river chart function debug

first_col='NormalizedName_author' 
first_col_filter='li feifei'
group_col='NormalizedName'
ptp_df=cvpr_graph_df 
backward=False
num_links=4








In [ ]:
# Bar chart function

from tqdm import tqdm
import matplotlib.pyplot as plt
import time

def plots_by_col_name(df, col_plot):
    '''
    Plot sum of patents with 'surveil' in them at different citation distances.
    '''
    
    # Consider only small subfields
    if col_plot=='NormalizedName_fields':
        df_e=df.explode([col_plot, 'PaperCount_fields'])
        df_e=df_e[(df_e['PaperCount_fields'].astype('float')<=250) & (df_e['PaperCount_fields'].astype('float')>0)]
    else:
        df_e=df.explode(col_plot)
    
    print('len df_e', len(df_e))
    # Only consider CVPR papers
    df_e=df_e[df_e['conf_if']=='1158167855'].reset_index()
    print('len df_e', len(df_e))
    
    uniques=df_e[col_plot].unique()
    print('num items in x axis', len(uniques))
    
    # Compute custom y axis, store in res_dict
    res_dict={col_plot:[], '0_dist':[], '1_dist':[], '2_dist':[], '3_dist':[], '0_other': [], '1_other': [], '2_other': [], '3_other': []}
    for unique in tqdm(uniques):
        res_dict[col_plot].append(unique)
        inds=df_e[df_e[col_plot] == unique].index
        for key in res_dict:
            if key!=col_plot:
                val=df_e[key][inds].sum()
                res_dict[key].append(val)

    # Make plots
    names_dict={'NormalizedName_fields': 'topic',
                'NormalizedName_author': 'author', 
                'NormalizedName': 'instituion', 
                'Iso3166Code': 'country'}

    df_e['0_dist']=df_e['0_dist']/(df_e['0_dist']+df_e['0_other'])
    df_e['1_dist']=df_e['1_dist']/(df_e['1_dist']+df_e['1_other'])
    df_e['2_dist']=df_e['2_dist']/(df_e['2_dist']+df_e['2_other'])
    df_e['3_dist']=df_e['3_dist']/(df_e['3_dist']+df_e['3_other'])
    
    for key in res_dict:
        if key!=col_plot:
            print('plot')
            df_plot=pd.DataFrame.from_dict(res_dict)
            df_plot=df_plot.sort_values(by=[key], ascending=False)
            # This is where the axes are choosen, only plot top 25
            ax=df_plot[:25].plot.bar(x=col_plot, y=key, title=f'Number Cites by Patents with "Surveil" by {names_dict[col_plot]} at cite distance {key}')
            ax.set_xlabel(names_dict[col_plot])
            ax.set_ylabel(f'Number Patents with "Surveil" in them at cite distance {key}')
            plt.show()
            
            
            

In [ ]:
# Make bar charts
cols_plot=['NormalizedName_fields','NormalizedName_author', 'NormalizedName', 'Iso3166Code']
for col_plot in cols_plot:
    plots_by_col_name(df, col_plot)

In [ ]:
# Debug/scratch
df_e=df.explode('NormalizedName_author')
df_e=df_e[df_e['paper_id']=='2155904486']
df_sw=df_e[df_e['NormalizedName_author']=='li feifei']
df_e=df_e.reset_index()
inds=df_e[df_e['NormalizedName_author'] == 'li feifei'].index.tolist()
print(inds)
print(df_e['0_dist'][inds])
print(df_e['0_dist'])
val=df_e['0_dist'][inds].sum()
print(val)
for i in range(len(df_sw.iloc[0])):
    print(df_sw.columns[i], df_sw.iloc[0,i])